# GBasis Tutorial - Basis Sets

# 1 Basis Sets

In [gbasis](https://gbasis.readthedocs.io/en/latest/), a basis set is defined as a list of [`GeneralizedContractionShell`](add_link) objects, each one representing a linear combination of primitives and containing all the information defining this combination (i.e the angular momentum of the shell, the center of the shell, the exponents of the primitives, the contraction coefficients, and the normalization coefficients).

This notebook showcases the different ways to define a basis set for a molecule and how to transform them.

## 1.1 Loading Basis Sets

Basis set information is often stored in text format. In `gbasis`, the Gaussian94 format (.gbs) and
the NWChem format (.nw) are supported.

In [1]:
import gbasis
import numpy as np
from gbasis.parsers import parse_gbs, parse_nwchem, make_contractions

# use H2 molecule as an example system
atoms = ["H", "H"]
coords = np.array([[0, 0, 0], [0, 0, 1]])

# load hydrogen def2-SVP basis set information with the Gaussian94 format
gbs_basis_dict = parse_gbs("hydrogen_def2-svp.1.gbs")
gbs_basis = make_contractions(gbs_basis_dict, atoms, coords)

# load hydrogen def2-SVP basis set information with the NWChem format
nw_basis_dict = parse_nwchem("hydrogen_def2-svp.1.nw")
nw_basis = make_contractions(nw_basis_dict, atoms, coords)

# print gbs_basis_dict and nw_basis_dict to verify they are the same
print("def2-SVP Basis Set Loaded from Gaussian94 Format:")
print(gbs_basis_dict)
print()
print("def2-SVP Basis Set Loaded from NWChem Format:")
print(nw_basis_dict)

def2-SVP Basis Set Loaded from Gaussian94 Format:
{'H': [(0, array([13.010701  ,  1.9622572 ,  0.44453796]), array([[0.01968216],
       [0.13796524],
       [0.47831935]])), (0, array([0.12194962]), array([[1.]])), (1, array([0.8]), array([[1.]]))]}

def2-SVP Basis Set Loaded from NWChem Format:
{'H': [(0, array([13.010701  ,  1.9622572 ,  0.44453796]), array([[0.01968216],
       [0.13796524],
       [0.47831935]])), (0, array([0.12194962]), array([[1.]])), (1, array([0.8]), array([[1.]]))]}


## 1.2 Building Basis Sets

`gbasis` interfaces to the `iodata` module, which handles the inputs and outputs for different quantum chemistry formats, such as Gaussian formatted checkpoint files (.fchk) and AIM
wavefunction files (.wfn and .wfx). Additonally, `gbasis` interfaces to `pyscf`, which is an ab initio computational chemistry program.

In [2]:
from iodata import load_one
from pyscf import gto
from gbasis.wrappers import from_iodata, from_pyscf

# build a basis from an fchk file (water at uwB97XD/def2-TZVPD LOT) using IOData
iodata_mol = load_one("water.fchk")
iodata_basis = from_iodata(iodata_mol)

# build an STO-3G basis for water using PySCF
pyscf_mol = gto.Mole()
pyscf_mol.build(
    atom = "O 0 0 0; H 0 1 0; H 0 0 1",
    basis = "sto-3g"
)
pyscf_basis = from_pyscf(pyscf_mol)

## 1.3 Types of Coordinate Systems Used by Basis Functions

In `gbasis`, the user can provide the coordinate system used by each shell of generalized contractions that gets stored in the basis list. The `make_contractions` function is useful for instantiating a basis for a system of atoms based on the `basis_dict` output of the parsers from `gbasis.parsers`. For a given call of `make_contractions`, one can pass either a string ("cartesian" or "spherical") or a list of strings to specify `coord_types`. If a string is passed, all contractions for all atoms are treated as having the same `coord_type`. 
If different shells correspond to different coordinate systems, then a list of the same length as the basis must be provided with each entry being "spherical" or "cartesian" to specify the coordinate system
of the corresponding shell. For example, if atom 1 contributes 8 contractions and atom 2 contributes 8 contractions, the `coord_types` list should be of length 16, where the first 8 elements correspond to contractions for atom 1 and the last 8 elements correspond to contractions for atom 2. These possibilities are illustrated in the code cell below using the Kr<sub>2</sub> molecule as an example:

In [3]:
# load STO-6G basis set information
basis_dict = parse_nwchem("data_sto6g.nwchem")

# create basis for Kr2 using cartesian coordinates for all generalized contractions
cartesian_basis = make_contractions(basis_dict, 
                                    ["Kr", "Kr"], 
                                    np.array([[0, 0, 0], [1, 0, 0]]),
                                    coord_types="cartesian")
print([shell.coord_type for shell in cartesian_basis])
print()

# create basis for Kr2 using spherical coordinates for all generalized contractions
spherical_basis = make_contractions(basis_dict, 
                                    ["Kr", "Kr"], 
                                    np.array([[0, 0, 0], [1, 0, 0]]),
                                    coord_types="spherical")
print([shell.coord_type for shell in spherical_basis])
print()

# create basis for Kr2 using spherical coordinates for all generalized contractions
# except the last one, which is specified to be cartesian
mixed_basis = make_contractions(basis_dict, 
                                ["Kr", "Kr"], 
                                np.array([[0, 0, 0], [1, 0, 0]]),
                                coord_types=["spherical"] * 15 + ["cartesian"])
print([shell.coord_type for shell in mixed_basis])

['cartesian', 'cartesian', 'cartesian', 'cartesian', 'cartesian', 'cartesian', 'cartesian', 'cartesian', 'cartesian', 'cartesian', 'cartesian', 'cartesian', 'cartesian', 'cartesian', 'cartesian', 'cartesian']

['spherical', 'spherical', 'spherical', 'spherical', 'spherical', 'spherical', 'spherical', 'spherical', 'spherical', 'spherical', 'spherical', 'spherical', 'spherical', 'spherical', 'spherical', 'spherical']

['spherical', 'spherical', 'spherical', 'spherical', 'spherical', 'spherical', 'spherical', 'spherical', 'spherical', 'spherical', 'spherical', 'spherical', 'spherical', 'spherical', 'spherical', 'cartesian']


## 1.4 Linear Transformations of Basis Functions

In `gbasis`, the user can linearly transform the basis functions before computing the desired properties.
All of the higher level functions have the keyword argument `transform` to specify the matrix that
transforms the basis set. The transformation is applied to the left, i.e.,

$$\begin{equation}
 \psi_i = \sum_j T_{ij} \phi_j
\end{equation}$$

where {$\phi_{j}$} is the basis set before transformation and {$\psi_{i}$} is the basis function after transformation.
The number of basis functions depends on the coordinate systems specified for each shell. 

Examples involving such transformations are provided in the tutorial notebooks on evaluations and integrals wherever molecular rather than atomic orbitals are involved in calculations.